In [1]:
file_path = './data/bankContent/all.txt'

data = ''
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        filename, txtdata = line.split('.txt ')[0], line.split('.txt ')[1] #資料剖析.split('.txt ')
        data += txtdata

#print(data)

In [2]:
file_path = './data/bankContent/cat_to_content_uncleaned.txt'

# Initialize a dictionary to hold the data
cat_to_content = {}

# Define the categories you're interested in
categories = ['g', 's', 'r', 'm']

# Open and read the file
file_path = './data/bankContent/cat_to_content_uncleaned.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Check which category the line belongs to
        for category in categories:
            if line.startswith(category):
                # If the category is found, add the data to the dictionary
                if category in cat_to_content:
                    cat_to_content[category] += line[len(category):].strip()  # Append the data without the category prefix #去除開頭和結尾的空白字符
                else:
                    cat_to_content[category] = line[len(category):].strip()  # Initialize the entry in the dictionary
                break  # Stop checking once the correct category is found

# Example output to check the structure
for category, data in cat_to_content.items():
    print(f"Category: {category}, Data: {data[:100]}...")  # Print the first 100 characters of the data for brevity
# Now `category_to_data` is a dictionary where each category maps to its corresponding text data.



Category: g, Data: 一、氣候治理(一)本行氣候治理架構及陳報流程鑒於氣候變遷之影響日益嚴重，且減少溫室氣體排放及強化氣候風險管理已成為全球趨勢，本行依循我國主管機關相關規範、本行「風險管理政策及指導準則」及「永續金融政策...
Category: s, Data: 二、氣候策略(一)氣候相關風險與機會辨識根據TCFD定義，氣候相關風險分為兩大類型，包括受氣候變遷影響之實體風險及轉型至低碳經濟之轉型風險。實體風險為因氣候變遷致極端天氣事件或氣候模式長期變化，所造成...
Category: r, Data: 3氣候風險管理31氣候風險管理政策本行為實踐永續金融發展目標，建立本行氣候風險管理制度及原則，提升氣候風險管理能力，制定有「氣候風險管理政策」，規範氣候風險組織架構及權責，以及建立氣候風險管理辨識、評...
Category: m, Data: 05指標與目標面對氣候變遷及極端氣候所引發的天災，花旗(台灣)以環境永續為主要思維，致力建構綠色低碳的企業環境。我們透過氣候風險熱圖及營運減碳管理，實踐愛地球的承諾。在本節中，我們總結相關量化資訊，展...


# Sentencize and tokenize

In [61]:
!pip install jieba
import jieba
import jieba.analyse
jieba.load_userdict('mydictionary.txt')
import re
import pandas as pd
from collections import defaultdict

In [63]:
data = cat_to_content['g']  # Get the data for the 'g' category

# 斷句
pattern = re.compile(r'([^。；！？]+)[。；！？]')  # 斷句#，
sentences = re.findall(pattern, data)
print(sentences[3:6])
#for sent in sentences[:10]:
#    print(sent)

# 斷詞
tokenized_sentences = []
for sent in sentences[3:6]:
    pattern = re.compile(r'[\u4e00-\u9fa5]')  # 只取中文
    sentr = pattern.findall(sent)
    sentr = ''.join(sentr)
    word = jieba.cut(sentr, cut_all=False)
    wordList = [w for w in word if len(w) >= 2]
    print(wordList)
    tokenized_sentences.append(wordList)
#print(tokenized_sentences)

['\uf06c監督氣候風險管理組織之運作以及目標之執行狀況', '\uf06c監督氣候風險情境分析之辦理情形及執行結果等', '風險管理委員會隸屬董事會，職責包括：\uf06c審議本行氣候風險相關政策與重要規章']
['監督', '氣候', '風險', '管理', '組織', '運作', '以及', '目標', '執行', '狀況']
['監督', '氣候', '風險', '情境', '分析', '辦理', '情形', '執行', '結果']
['風險', '管理', '委員會', '隸屬', '董事會', '職責', '包括', '審議', '本行', '氣候', '風險', '相關', '政策', '重要', '規章']


# Restore the sentence, distance and co-occurence weight of the target word and related word

In [65]:
# 設定先行詞
target_word = '風險'
# 初始化一個字典來存儲結果
co_occurrence_dict = defaultdict(list)

# 計算每個句子中先行詞和關聯詞的距離與共現權重
for idx, wordList in enumerate(tokenized_sentences):
    print(f"id:{idx}, wordList:{wordList}")
    last_index = -1  # 新句子初始化為-1，表示尚未找到先行詞

    # 處理1~n-1先行詞到句尾
    for i, word in enumerate(wordList):
        #print(f"id:{idx}, last_index:{last_index}, i:{i}, word:{word}")
        if word == target_word:
            if last_index != -1:  # 找到先行詞
                for j in range(last_index + 1, i):
                    related_word = wordList[j] 
                    distance = sum(len(wordList[k]) for k in range(last_index + 1, j))
                    weight = 1 / (distance + 1)
                    print(f"id:{idx}, 先行詞: {target_word}, 關聯詞: {related_word}, 距離: {distance}, 共現權重: {weight:.3f}")
                    co_occurrence_dict[(target_word, related_word)].append([idx, distance, weight])
                    
            last_index = i # 更新到下個先行詞的位置

    # 處理第n個先行詞到句尾
    if last_index != -1:
        for j in range(last_index + 1, len(wordList)):
            related_word = wordList[j]
            distance = sum(len(wordList[k]) for k in range(last_index + 1, j))
            weight = 1 / (distance + 1)
            print(f"id:{idx}, 先行詞: {target_word}, 關聯詞: {related_word}, 距離: {distance}, 共現權重: {weight:.3f}")
            co_occurrence_dict[(target_word, related_word)].append([idx, distance, weight])
print(co_occurrence_dict)

id:0, wordList:['監督', '氣候', '風險', '管理', '組織', '運作', '以及', '目標', '執行', '狀況']
id:0, 先行詞: 風險, 關聯詞: 管理, 距離: 0, 共現權重: 1.000
id:0, 先行詞: 風險, 關聯詞: 組織, 距離: 2, 共現權重: 0.333
id:0, 先行詞: 風險, 關聯詞: 運作, 距離: 4, 共現權重: 0.200
id:0, 先行詞: 風險, 關聯詞: 以及, 距離: 6, 共現權重: 0.143
id:0, 先行詞: 風險, 關聯詞: 目標, 距離: 8, 共現權重: 0.111
id:0, 先行詞: 風險, 關聯詞: 執行, 距離: 10, 共現權重: 0.091
id:0, 先行詞: 風險, 關聯詞: 狀況, 距離: 12, 共現權重: 0.077
id:1, wordList:['監督', '氣候', '風險', '情境', '分析', '辦理', '情形', '執行', '結果']
id:1, 先行詞: 風險, 關聯詞: 情境, 距離: 0, 共現權重: 1.000
id:1, 先行詞: 風險, 關聯詞: 分析, 距離: 2, 共現權重: 0.333
id:1, 先行詞: 風險, 關聯詞: 辦理, 距離: 4, 共現權重: 0.200
id:1, 先行詞: 風險, 關聯詞: 情形, 距離: 6, 共現權重: 0.143
id:1, 先行詞: 風險, 關聯詞: 執行, 距離: 8, 共現權重: 0.111
id:1, 先行詞: 風險, 關聯詞: 結果, 距離: 10, 共現權重: 0.091
id:2, wordList:['風險', '管理', '委員會', '隸屬', '董事會', '職責', '包括', '審議', '本行', '氣候', '風險', '相關', '政策', '重要', '規章']
id:2, 先行詞: 風險, 關聯詞: 管理, 距離: 0, 共現權重: 1.000
id:2, 先行詞: 風險, 關聯詞: 委員會, 距離: 2, 共現權重: 0.333
id:2, 先行詞: 風險, 關聯詞: 隸屬, 距離: 5, 共現權重: 0.167
id:2, 先行詞: 風險, 關聯詞: 董事會, 距離: 7, 共現權重: 0.125
id:2, 先行詞:

In [ ]:
trigram_dict = defaultdict(list)

# 遍歷每個句子
for idx, wordList in enumerate(tokenized_sentences): 
    for i, word in enumerate(wordList):
        #print(f"id:{idx}, wordList:{wordList}")
        if word == target_word:
            # 找到二連詞的第二個詞（related_word）
            for j in range(i + 1, len(wordList)):
                related_word = wordList[j]
                # 計算距離
                distance_1 = sum(len(wordList[k]) for k in range(i + 1, j))
                #print(f"id:{idx}, 先行詞: {target_word}, 關聯詞一: {related_word}, 距離一: {distance_1}")
                if j + 1 < len(wordList):
                    # 找到三連詞的第三個詞（third_word）
                    for k in range(j + 1, len(wordList)):
                        third_word = wordList[k]
                        # 計算距離
                        distance_2 = sum(len(wordList[l]) for l in range(j + 1, k))
                        #print(f"id:{idx}, 先行詞: {target_word}, 關聯詞一: {related_word}, 關聯詞二: {third_word}, 距離二: {distance_2}")
                        total_distance = distance_1 + distance_2
                        weight = 1 / (total_distance + 1)
                        print(f"id:{idx}, 先行詞: {target_word}, 關聯詞一: {related_word}, 關聯詞二: {third_word}, 距離: {total_distance}, 共現權重: {weight:.3f}")
                        # 記錄三連詞
                        trigram_dict[(target_word, related_word, third_word)].append([idx, total_distance, weight])
                        

In [ ]:
# 輸出結果
for key, values in co_occurrence_dict.items():
    print(len(values))
    print(f"{key} : {values}")


# Calculate co-occurrence frequency and co-occurrence weight

In [ ]:
# 計算每個先行詞和關聯詞的平均共現權重
total_occurrences = 0

for key, values in trigram_dict.items():
    occurrence_count = len(values)
    total_occurrences += occurrence_count
print(total_occurrences)


co_occurrence_dict_avg = {}
for key, values in trigram_dict.items():
    avg_weight = sum([w[2] for w in values]) / len(values)
    co_occurrence_dict_avg[key] = avg_weight
    occurrence_count = len(values)
    occurrence_frequency = occurrence_count / total_occurrences
    co_occurrence_dict_avg[key] = avg_weight, occurrence_count, occurrence_frequency
print(co_occurrence_dict_avg)
print("平均共現權重：")
for key, (avg_weight, occurrence_count, occurrence_frequency) in co_occurrence_dict_avg.items():
    print(f"{key} : 平均共現權重：{avg_weight:.3f}, 共現次數：{occurrence_count}, 共現頻率：{occurrence_frequency:.3f}")

In [ ]:
# 將結果轉換為DataFrame
df = pd.DataFrame([
    {"先行詞": key[0], "關聯詞": key[1], "平均共現權重": avg_weight, "共現次數": occurrence_count, "共現頻率": occurrence_frequency} 
    for key, (avg_weight, occurrence_count, occurrence_frequency) in co_occurrence_dict_avg.items()
])

df = df.sort_values(by="平均共現權重", ascending=False)
# 重置索引
df.reset_index(drop=True, inplace=True)
print(df)

df.to_excel('co_occurrence_weights.xlsx', index=False)

# Process

In [7]:
!pip install jieba
import jieba
import jieba.analyse
jieba.load_userdict('mydictionary.txt')
import re
import pandas as pd
from collections import defaultdict
import re

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\USER\AppData\Local\Temp\jieba.cache
Loading model cost 0.450 seconds.
Prefix dict has been built successfully.


In [8]:
def sentencize(data):
    import re
    pattern = re.compile(r'([^。；！？]+)[。；！？]')  # 斷句#，
    sentences = re.findall(pattern, data)
    #print(sentences[:10])
    return sentences

In [9]:
def tokenize(sentences):
    # 斷詞
    tokenized_sentences = []
    for sent in sentences:
        pattern = re.compile(r'[\u4e00-\u9fa5]')  # 只取中文
        sentr = pattern.findall(sent)
        sentr = ''.join(sentr)
        word = jieba.cut(sentr, cut_all=False)
        wordList = [w for w in word if len(w) >= 2]
        #print(wordList)
        tokenized_sentences.append(wordList)
    #print(tokenized_sentences)
    return tokenized_sentences

In [10]:
def co_occurrence_dict(target_word, tokenized_sentences):
    # 設定先行詞
    #target_word = '氣候'
    # 初始化一個字典來存儲結果
    co_occurrence_dict = defaultdict(list)

    # 計算每個句子中先行詞和關聯詞的距離與共現權重
    for idx, wordList in enumerate(tokenized_sentences):
        #print(f"id:{idx}, wordList:{wordList}")
        last_index = -1  # 新句子初始化為-1，表示尚未找到先行詞

        # 處理1~n-1先行詞到句尾
        for i, word in enumerate(wordList):
            #print(f"id:{idx}, last_index:{last_index}, i:{i}, word:{word}")
            if word == target_word:
                if last_index != -1:  # 找到先行詞
                    for j in range(last_index + 1, i):
                        related_word = wordList[j] 
                        distance = sum(len(wordList[k]) for k in range(last_index + 1, j))
                        weight = 1 / (distance + 1)
                        #print(f"id:{idx}, 先行詞: {target_word}, 關聯詞: {related_word}, 距離: {distance}, 共現權重: {weight:.3f}")
                        co_occurrence_dict[(target_word, related_word)].append([idx, distance, weight])
                        
                last_index = i # 更新到下個先行詞的位置

        # 處理第n個先行詞到句尾
        if last_index != -1:
            for j in range(last_index + 1, len(wordList)):
                related_word = wordList[j]
                distance = sum(len(wordList[k]) for k in range(last_index + 1, j))
                weight = 1 / (distance + 1)
                #print(f"id:{idx}, 先行詞: {target_word}, 關聯詞: {related_word}, 距離: {distance}, 共現權重: {weight:.3f}")
                co_occurrence_dict[(target_word, related_word)].append([idx, distance, weight])
    #print(co_occurrence_dict)
    return co_occurrence_dict

In [11]:
def calculate_co_occurrence_dict_avg(co_occurrence_dict):
    # 計算二連詞，每個先行詞和關聯詞的平均共現權重
    total_occurrences = 0

    for key, values in co_occurrence_dict.items():
        occurrence_count = len(values)
        total_occurrences += occurrence_count
    #print(total_occurrences)


    co_occurrence_dict_avg = {}
    for key, values in co_occurrence_dict.items():
        avg_weight = sum([w[2] for w in values]) / len(values)
        co_occurrence_dict_avg[key] = avg_weight
        occurrence_count = len(values)
        occurrence_frequency = occurrence_count / total_occurrences
        co_occurrence_dict_avg[key] = avg_weight, occurrence_count, occurrence_frequency
    #print(co_occurrence_dict_avg)
    #print("平均共現權重：")
    for key, (avg_weight, occurrence_count, occurrence_frequency) in co_occurrence_dict_avg.items():
        #print(f"{key} : 平均共現權重：{avg_weight:.3f}, 共現次數：{occurrence_count}, 共現頻率：{occurrence_frequency:.3f}")
        return co_occurrence_dict_avg

In [12]:
def dict_to_df(target_word, cat, dict):
    # 將結果轉換為DataFrame
    df = pd.DataFrame([
        {"先行詞": key[0], "關聯詞": key[1], "平均共現權重": avg_weight, "共現次數": occurrence_count, "共現頻率": occurrence_frequency} 
        for key, (avg_weight, occurrence_count, occurrence_frequency) in dict.items()
    ])

    df = df.sort_values(by=["共現次數", "平均共現權重"], ascending=False)
    # 重置索引
    df.reset_index(drop=True, inplace=True)

    #df.to_excel(f'./association_words/{cat}/{target_word}_co_occurrence_dict_avg_{cat}.xlsx', index=False)
    #df.to_excel(f'./association_words/life/{cat}/{target_word}_co_occurrence_dict_avg_{cat}.xlsx', index=False)
    df.to_excel(f'./association_words/property/{cat}/{target_word}_co_occurrence_dict_avg_{cat}.xlsx', index=False)
    return df

In [13]:
def co_occurrence_dict_third(target_word, tokenized_sentences):
    #計算三連詞
    trigram_dict = defaultdict(list)

    # 遍歷每個句子
    for idx, wordList in enumerate(tokenized_sentences): 
        for i, word in enumerate(wordList):
            #print(f"id:{idx}, wordList:{wordList}")
            if word == target_word:
                # 找到二連詞的第二個詞（related_word）
                for j in range(i + 1, len(wordList)):
                    related_word = wordList[j]
                    # 計算距離
                    distance_1 = sum(len(wordList[k]) for k in range(i + 1, j))
                    #print(f"id:{idx}, 先行詞: {target_word}, 關聯詞一: {related_word}, 距離一: {distance_1}")
                    if j + 1 < len(wordList):
                        # 找到三連詞的第三個詞（third_word）
                        for k in range(j + 1, len(wordList)):
                            third_word = wordList[k]
                            # 計算距離
                            distance_2 = sum(len(wordList[l]) for l in range(j + 1, k))
                            #print(f"id:{idx}, 先行詞: {target_word}, 關聯詞一: {related_word}, 關聯詞二: {third_word}, 距離二: {distance_2}")
                            total_distance = distance_1 + distance_2
                            weight = 1 / (total_distance + 1)
                            #print(f"id:{idx}, 先行詞: {target_word}, 關聯詞一: {related_word}, 關聯詞二: {third_word}, 距離: {total_distance}, 共現權重: {weight:.3f}")
                            # 記錄三連詞
                            trigram_dict[(target_word, related_word, third_word)].append([idx, total_distance, weight])
    return trigram_dict                        

In [14]:
def dict_third_to_df(target_word, cat, dict_third):
    # 將結果轉換為DataFrame
    df = pd.DataFrame([
        {"先行詞": key[0], "關聯詞一": key[1], "關聯詞二": key[2], "平均共現權重": avg_weight, "共現次數": occurrence_count, "共現頻率": occurrence_frequency} 
        for key, (avg_weight, occurrence_count, occurrence_frequency) in dict_third.items()
    ])

    df = df.sort_values(by=["共現次數", "平均共現權重"], ascending=False)
    # 重置索引
    df.reset_index(drop=True, inplace=True)

    #df.to_excel(f'./association_words/{cat}/{target_word}_co_occurrence_dict_third_avg_{cat}.xlsx', index=False)
    #df.to_excel(f'./association_words/life/{cat}/{target_word}_co_occurrence_dict_third_avg_{cat}.xlsx', index=False)
    df.to_excel(f'./association_words/property/{cat}/{target_word}_co_occurrence_dict_third_avg_{cat}.xlsx', index=False)
    return df

In [15]:
def merge_and_sort_dataframes(df_list, sort_by=["共現次數", "平均共現權重"], ascending=[False, False]):
    """
    將多個DataFrame縱向合併，並根據指定的欄位進行排序
    df_list: 要合併的DataFrame列表
    sort_by: 用來排序的欄位列表，默認是 ["共現次數", "平均共現權重"]
    ascending: 排序順序，默認是降冪排序 [False, False]
    """
    # 縱向合併多個DataFrame
    merged_df = pd.concat(df_list, ignore_index=True)
    
    # 根據指定的欄位進行排序
    merged_df = merged_df.sort_values(by=sort_by, ascending=ascending)
    
    # 重置索引
    merged_df.reset_index(drop=True, inplace=True)
    
    return merged_df

In [16]:
category_to_data = {}

# 將category列表中的每個項目初始化到字典中
categories = ['g', 's', 'r', 'm']
for category in categories:
    category_to_data[category] = ""

# 打開文件並逐行處理
#with open('./data/bankContent/cat_to_content_uncleaned.txt', 'r', encoding='utf-8') as file:
#with open('./data/insuranceContent/l/cat_to_content_uncleaned_life.txt', 'r', encoding='utf-8') as file:
with open('./data/insuranceContent/p/cat_to_content_uncleaned_property.txt', 'r', encoding='utf-8') as file:
    current_category = None
    for line in file:
        line = line.strip()  # 去掉每行的換行符和首尾的空白字符
        if not line:  # 跳過空行
            continue
        #if line[0] == 'g':
        if line[0] in categories:  # 如果該行的第一個字符是類別標記
            current_category = line[0]  # 更新當前的類別
            category_to_data[current_category] += line[1:].strip()  # 將該行的內容加入對應類別中
        elif current_category:  # 如果當前類別已經設置
            category_to_data[current_category] += line.strip()  # 將內容添加到當前類別的數據中

print(category_to_data.keys())
print(category_to_data)
print(category_to_data['g'][:100])

dict_keys(['g', 's', 'r', 'm'])
{'g': "治理氣候治理架構隨著全球氣候變遷意識上升，和泰產險意識到企業正面臨著前所未有的風險與挑戰。為確保企業能夠永續且穩定發展，和泰產險建立有效的氣候治理架構，透過董事會、企業永續委員會、風險管理委員會、權責單位及各業務單位由上而下推動，針對氣候變遷風險制定相應的管理措施以減緩和調適氣候變遷帶來的影響，並確保企業能夠在變化的環境中保持穩定和發展。\uf0b2董事會和泰產險董事會為氣候相關議題的最高治理單位，負責核定風險胃納，對氣候相關風險與機會之管理負有最終責任。\uf0b2企業永續委員會董事會轄下設立「企業永續委員會」，由董事長擔任主任委員，總經理為總幹事，風險管理部為事務局，組成小組包括綠色商品小組、永續營運小組、責任投資小組、社會責任小組、永續治理小組及TCFD小組。委員會負責監督及指導氣候相關風險與機會之管理情形，每年至少召開二次會議，得視需要隨時召開會議，並定期向董事會報告執行成果。\uf0b2風險管理委員會董事會轄下設立「風險管理委員會」，由兩名董事與風控長共同組成，獨立董事擔任召集人，負責監督及指導本公司氣候相關風險管理執行情形。委員會原則上每季召開會議，並定期向董事會報告並適時反應風險管理執行情形，提出必要之改善建議。\uf0b2風險管理部風險管理部規劃氣候相關風險管理機制，確保相關業務單位已適當管理氣候相關風險，逐步發展氣候相關風險情境分析與壓力測試，彙整氣候相關風險管理推動情形，並定期向風險管理委員會報告。\uf0b2業務單位各業務單位其業務內容涉及氣候變遷相關者，應依公司相關政策及管理規範執行，充分瞭解所負責業務之氣候相關風險與機會，鑑別其對財務、業務、商品及投資等之財務影響，並落實執行氣候變遷相關管理程序。強化氣候風險知識為強化和泰產險氣候韌性管理，和泰產險委請外部顧問為公司董監事及管理階層進行氣候相關教育訓練，2022年度本公司全體董事及高階主管皆參與氣候變遷發展趨勢與TCFD簡介教育訓練，訓練時數3小時，未來仍持續規劃氣候變遷相關風險與機會的教育訓練，提升董事及員工對於氣候變遷風險的認識。Chapter1氣候治理隨著極端天氣與氣候變遷日漸嚴峻，各國政府及企業應更積極思考如何共同抑制全球升溫議題。2022年各國領導人於埃及夏姆錫克(Sharmel-Sheikh)舉行聯合國

##  Recursively go through all the words of each element, identify the target word, related word one(二連詞), and related word two(三連詞)

In [23]:
df_target_words = pd.read_excel('./word_frequency/property_m_freq.xlsx')
target_words = df_target_words['word'].tolist()
target_words[:10]


['目標', '排放', '公司', '溫室氣體', '氣候', '指標', '管理', '投資', '風險', '組合']

In [24]:
df_list = []  # 用來存放所有的df
df_third_list = []  # 用來存放所有的df_third

#target_word = '氣候'
for word in target_words:#[:2]
    #print(word)
    target_word = word
    for category, data in category_to_data.items():
        #print(f"Category: {category}")
        if category == 'm':
            sentences = sentencize(category_to_data[category])
            tokenized_sentences = tokenize(sentences)
            co_occurrence_dict_info = co_occurrence_dict(target_word, tokenized_sentences)
            co_occurrence_dict_avg = calculate_co_occurrence_dict_avg(co_occurrence_dict_info)
            if co_occurrence_dict_avg is None:
                continue
            df = dict_to_df(target_word, category, co_occurrence_dict_avg)
            print(df)
            df_list.append(df)  # 將df加入列表

            co_occurrence_dict_third_info = co_occurrence_dict_third(target_word, tokenized_sentences)
            co_occurrence_dict_third_avg = calculate_co_occurrence_dict_avg(co_occurrence_dict_third_info)
            if co_occurrence_dict_third_avg is None:
                continue
            df_third = dict_third_to_df(target_word, category, co_occurrence_dict_third_avg)
            print(df_third)
            df_third_list.append(df_third)

    先行詞 關聯詞    平均共現權重  共現次數      共現頻率
0    目標  氣候  0.178228    42  0.028610
1    目標  風險  0.073579    32  0.021798
2    目標  公司  0.118855    26  0.017711
3    目標  管理  0.183236    24  0.016349
4    目標  相關  0.077327    23  0.015668
..   ..  ..       ...   ...       ...
649  目標  智慧  0.010101     1  0.000681
650  目標  機械  0.009901     1  0.000681
651  目標  韌性  0.009434     1  0.000681
652  目標  基金  0.009346     1  0.000681
653  目標  方向  0.009091     1  0.000681

[654 rows x 5 columns]
      先行詞 關聯詞一 關聯詞二    平均共現權重  共現次數      共現頻率
0      目標   氣候   目標  0.035258    53  0.001162
1      目標   公司   目標  0.013341    46  0.001009
2      目標   目標   目標  0.014233    42  0.000921
3      目標   設定   目標  0.053934    41  0.000899
4      目標   氣候   風險  0.066074    40  0.000877
...    ..  ...  ...       ...   ...       ...
21203  目標  排放量   客戶  0.003623     1  0.000022
21204  目標   依據   客戶  0.003610     1  0.000022
21205  目標   庫數   客戶  0.003610     1  0.000022
21206  目標   組合   客戶  0.003610     1  0.000022
21207  目標   中長

## Merge results of 二連詞或三連詞 for each element

In [25]:
# 定義要儲存的Excel最大行數
MAX_ROWS = 1048575

merged_df = merge_and_sort_dataframes(df_list)
merged_df_third = merge_and_sort_dataframes(df_third_list)

# 將合併後的結果存儲到Excel
merged_df.iloc[:MAX_ROWS, :].to_excel('./association_words/property/m/merged_co_occurrence_dict_avg_m.xlsx', index=False)
merged_df_third.iloc[:MAX_ROWS, :].to_excel('./association_words/property/m/merged_co_occurrence_dict_third_avg_m.xlsx', index=False)

print(merged_df.head(20))
print(merged_df_third.head(20))

     先行詞   關聯詞    平均共現權重  共現次數      共現頻率
0     氣候    風險  0.338933   210  0.069559
1     相關    風險  0.543765   150  0.070028
2     風險    管理  0.767456   118  0.035899
3   氣候變遷    風險  0.302002   118  0.058243
4     氣候    相關  0.776728   114  0.037761
5     公司    風險  0.147225   109  0.038045
6     風險    氣候  0.138335    95  0.028902
7     管理    風險  0.093939    94  0.055294
8     氣候    管理  0.165121    84  0.027824
9     風險    相關  0.138608    75  0.022817
10    相關    管理  0.153050    66  0.030812
11    相關    氣候  0.069551    62  0.028945
12    風險    公司  0.150686    57  0.017341
13  氣候變遷    公司  0.160244    56  0.027641
14    公司    相關  0.096674    56  0.019546
15    評估    風險  0.091558    56  0.055227
16    風險  氣候變遷  0.160309    55  0.016733
17    轉型    風險  0.400019    54  0.068615
18    指標    目標  0.644367    53  0.051758
19  氣候變遷    相關  0.264174    53  0.026160
     先行詞  關聯詞一 關聯詞二    平均共現權重  共現次數      共現頻率
0     風險    風險   風險  0.029570   640  0.005008
1     氣候    風險   風險  0.029197   425  0.004641
2

## Merge results of all elements

In [26]:
import pandas as pd
import glob
# 讀取所有 Excel 檔案
excel_files = glob.glob('./association_words/property/merged_co_occurrence_dict_third_avg_*.xlsx')  # 根據你的文件夾結構調整路徑

df_third_list = []  # 用來存放所有的df

# 讀取每個 Excel 文件並將其添加到 df_list
for file in excel_files:
    print(file)
    df = pd.read_excel(file)
    df_third_list.append(df)


./association_words/property\merged_co_occurrence_dict_third_avg_s.xlsx
./association_words/property\merged_co_occurrence_dict_third_avg_g.xlsx
./association_words/property\merged_co_occurrence_dict_third_avg_r.xlsx
./association_words/property\merged_co_occurrence_dict_third_avg_m.xlsx


In [27]:
excel_files = glob.glob('./association_words/property/merged_co_occurrence_dict_avg_*.xlsx')  # 根據你的文件夾結構調整路徑

df_list = []  # 用來存放所有的df

# 讀取每個 Excel 文件並將其添加到 df_list
for file in excel_files:
    print(file)
    df = pd.read_excel(file)
    df_list.append(df)

./association_words/property\merged_co_occurrence_dict_avg_s.xlsx
./association_words/property\merged_co_occurrence_dict_avg_g.xlsx
./association_words/property\merged_co_occurrence_dict_avg_r.xlsx
./association_words/property\merged_co_occurrence_dict_avg_m.xlsx


In [28]:
print(df_third_list[:1])

[         先行詞 關聯詞一   關聯詞二    平均共現權重  共現次數      共現頻率
0         風險   風險     風險  0.026294  1117  0.005491
1         氣候   氣候     氣候  0.010887  1035  0.003881
2         氣候   風險     風險  0.027466  1003  0.003761
3         氣候   氣候     風險  0.020297   952  0.003570
4         風險   氣候     風險  0.030750   855  0.004203
...      ...  ...    ...       ...   ...       ...
1048570  並採用   轉型  對華南產險  0.026316     1  0.000393
1048571  投資人   盡職     互動  0.026316     1  0.004831
1048572  投資人   治理     互動  0.026316     1  0.004831
1048573  投資人   透過     互動  0.026316     1  0.004831
1048574  投資人   電話     互動  0.026316     1  0.004831

[1048575 rows x 6 columns]]


In [29]:
MAX_ROWS = 1048575

merged_df = merge_and_sort_dataframes(df_list)
merged_df_third = merge_and_sort_dataframes(df_third_list)

# 將合併後的結果存儲到Excel
merged_df.iloc[:MAX_ROWS, :].to_excel('./association_words/property/merged_co_occurrence_dict_avg.xlsx', index=False) #merged_co_occurrence_dict_third_avg_*合併
merged_df_third.iloc[:MAX_ROWS, :].to_excel('./association_words/property/merged_co_occurrence_dict_third_avg.xlsx', index=False)

print(merged_df.head(20))
print(merged_df_third.head(20))

     先行詞 關聯詞    平均共現權重  共現次數      共現頻率
0     氣候  風險  0.425772   338  0.118721
1     氣候  風險  0.478502   328  0.085328
2     管理  風險  0.160328   269  0.103065
3     風險  管理  0.786680   247  0.058159
4     氣候  風險  0.338933   210  0.069559
5     相關  風險  0.482948   204  0.115319
6     公司  風險  0.126583   181  0.089826
7     相關  風險  0.502756   178  0.085127
8     風險  氣候  0.101201   176  0.041431
9     相關  風險  0.543765   150  0.070028
10    風險  氣候  0.123736   136  0.032023
11    氣候  相關  0.839074   134  0.034860
12    氣候  管理  0.165351   134  0.047067
13    氣候  風險  0.467988   126  0.086897
14    氣候  相關  0.793266   119  0.041798
15    風險  管理  0.767456   118  0.035899
16  氣候變遷  風險  0.302002   118  0.058243
17    評估  風險  0.078597   117  0.063934
18    管理  風險  0.104379   116  0.081863
19    氣候  相關  0.776728   114  0.037761
   先行詞 關聯詞一 關聯詞二    平均共現權重  共現次數      共現頻率
0   風險   風險   風險  0.027352  2265  0.011124
1   氣候   風險   風險  0.029021  1344  0.014143
2   風險   風險   管理  0.034887  1303  0.006399
3   風險   